In [12]:
import os
import gzip
import tempfile
import random
import pickle

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_model_optimization as tfmot

def get_gzipped_model_size(file):
    # Returns the size of the gzipped model in bytes.
    _, zipped_file = tempfile.mkstemp('.zip')
    with gzip.open(zipped_file, 'wb') as f:
        with open(file, 'rb') as model_file:
            f.write(model_file.read())
    return os.path.getsize(zipped_file)

def representative_data_gen():
    """Generator function for representative data required for TFLite conversion."""
    for input_value in tf.data.Dataset.from_tensor_slices(X_train.astype(np.float32)).batch(1).take(100):
        yield [input_value]

def divide_samples(data, states, division):
    """Divide the data samples for more granular analysis."""
    divisions = division
    new_data_shape = (data.shape[0] * divisions, int(data.shape[1] / divisions))
    new_data = data.reshape(new_data_shape)
    new_states = np.repeat(states, divisions)
    return new_data, new_states

def evaluate_tflite_model(tflite_model, X, y):
    # Initialize the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    input_index = interpreter.get_input_details()[0]['index']
    output_index = interpreter.get_output_details()[0]['index']

    # Iterate over the test data and make predictions
    predictions = []
    for test_sample in X:
        test_sample = np.expand_dims(test_sample, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_sample)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_index)
        predictions.append(output_data[0])

    # Convert predictions to the same format as y for comparison
    predictions = np.argmax(predictions, axis=1)
    if y.ndim > 1:  # Assuming y is one-hot encoded
        y = np.argmax(y, axis=1)

    return accuracy_score(y, predictions)

In [13]:
subj = 'subject1'
path = './data'

import numpy as np
import pickle

# Load SleepData
SleepData = np.load(f"{path}/SleepData_{subj}.npy")
with open(f"{path}/SleepData_metadata_{subj}.pkl", 'rb') as f:
    SleepData_metadata = pickle.load(f)

# Load Data
Data = np.load(f"{path}/Data_{subj}.npy")
with open(f"{path}/Data_metadata_{subj}.pkl", 'rb') as f:
    Data_metadata = pickle.load(f)

# Print shapes and unique states for SleepData
print("SleepData Shape:", SleepData.shape)
print("Unique States in SleepData:", np.unique(SleepData_metadata['GT']))

# Print shapes and unique states for Data
print("Data Shape:", Data.shape)
print("Unique States in Data:", np.unique(Data_metadata['GT']))

# 'MA' is encoded as 0 and 'slow_updown' as 1
labels_SleepData = np.array([0 if state == 'MA' else 1 for state in SleepData_metadata['GT']])

# Encoding 'Awake' as 0 and 'Sleep' as 1
labels_Data = np.array([1 if state == 'Sleep' else 0 for state in Data_metadata['GT']])




SleepData Shape: (576, 2000)
Unique States in SleepData: ['MA' 'slow_updown']
Data Shape: (864, 2000)
Unique States in Data: ['Sleep' 'awake']


In [14]:
def divide_samples(data, states,division):
    # Number of divisions per sample
    divisions = division
    # New shape for data
    new_data_shape = (data.shape[0] * divisions, int(data.shape[1] / divisions))
    # Reshape data
    new_data = data.reshape(new_data_shape)
    # Repeat states to match the new number of samples
    new_states = np.repeat(states, divisions)
    return new_data, new_states

division = 10
subset_data, subset_states = divide_samples(SleepData, labels_SleepData,division)
X = subset_data
y = subset_states
num_classes = 2
label_encoder = LabelEncoder()
y_integers = label_encoder.fit_transform(y)
states_encoded = to_categorical(y_integers, num_classes=num_classes)
X_train, X_test, y_train, y_test = train_test_split(X, states_encoded, test_size=0.2,
                                                                    random_state=42, stratify=states_encoded)
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(4608, 200)
(4608, 2)
(1152, 200)
(1152, 2)


In [15]:
model = Sequential([
    Dense(10, activation='relu', input_shape=(200,)),  # Adjusted input shape
    Dense(len(np.unique(states_encoded)), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=15, batch_size=128)

y_test_integers = np.argmax(y_test, axis=1)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
original_accuracy = accuracy_score(y_test_integers, y_pred_classes)


# Your model conversion and evaluation code
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]
quantized_tflite_model = converter.convert()

# Evaluate TFLite model
quantized_accuracy = evaluate_tflite_model(quantized_tflite_model, X_test, y_test)

# Print accuracies
print("Original Model Accuracy: {:.2f}%".format(original_accuracy * 100))
print("Quantized Model Accuracy: {:.2f}%".format(quantized_accuracy * 100))

# Save the original model
model.save('model_sleep.h5')
original_model_size = get_gzipped_model_size('model_sleep.h5')
print("Size of gzipped original Keras model: %.2f Kbytes" % (original_model_size / 1024))

# Save the TFLite model
tflite_model_path = 'quantized_f32_model_sleep.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(quantized_tflite_model)

# Calculate and print the size of the TFLite model
quantized_pruned_model_size = get_gzipped_model_size(tflite_model_path)
print("Size of gzipped quantized TFLite model: %.2f Kbytes" % (quantized_pruned_model_size / 1024))




Epoch 1/15
36/36 [==============================] - 0s 719us/step - loss: 0.6665 - accuracy: 0.6157
Epoch 2/15
36/36 [==============================] - 0s 650us/step - loss: 0.6099 - accuracy: 0.6879
Epoch 3/15
36/36 [==============================] - 0s 586us/step - loss: 0.5887 - accuracy: 0.7010
Epoch 4/15
36/36 [==============================] - 0s 504us/step - loss: 0.5808 - accuracy: 0.7055
Epoch 5/15
36/36 [==============================] - 0s 526us/step - loss: 0.5771 - accuracy: 0.7079
Epoch 6/15
36/36 [==============================] - 0s 520us/step - loss: 0.5723 - accuracy: 0.7166
Epoch 7/15
36/36 [==============================] - 0s 546us/step - loss: 0.5662 - accuracy: 0.7127
Epoch 8/15
36/36 [==============================] - 0s 530us/step - loss: 0.5617 - accuracy: 0.7244
Epoch 9/15
36/36 [==============================] - 0s 509us/step - loss: 0.5552 - accuracy: 0.7305
Epoch 10/15
36/36 [==============================] - 0s 517us/step - loss: 0.5505 - accuracy: 0.7337

INFO:tensorflow:Assets written to: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpu44t416i/assets


INFO:tensorflow:Assets written to: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpu44t416i/assets


Original Model Accuracy: 75.09%
Quantized Model Accuracy: 75.09%
Size of gzipped original Keras model: 24.77 Kbytes
Size of gzipped quantized TFLite model: 8.27 Kbytes


2023-12-11 00:47:31.996160: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-11 00:47:31.996176: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-12-11 00:47:31.996282: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpu44t416i
2023-12-11 00:47:31.997032: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-12-11 00:47:31.997040: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpu44t416i
2023-12-11 00:47:31.999531: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-12-11 00:47:32.026694: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpu44t416i
2023-12-

In [16]:
## now i

## now Data from Awake vs. Sleep

In [17]:
division = 10
subset_data_data, subset_states_data = divide_samples(Data, labels_Data, division)
X = subset_data_data
y = subset_states_data
num_classes = 2
label_encoder = LabelEncoder()
y_integers = label_encoder.fit_transform(y)
states_encoded = to_categorical(y_integers, num_classes=num_classes)
X_train, X_test, y_train, y_test = train_test_split(X, states_encoded, test_size=0.2,
                                                                    random_state=42, stratify=states_encoded)
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(6912, 200)
(6912, 2)
(1728, 200)
(1728, 2)


In [18]:
model = Sequential([
    Dense(10, activation='relu', input_shape=(200,)),  # Adjusted input shape
    Dense(len(np.unique(states_encoded)), activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=15, batch_size=128)

y_test_integers = np.argmax(y_test, axis=1)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
original_accuracy = accuracy_score(y_test_integers, y_pred_classes)


# Your model conversion and evaluation code
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_types = [tf.float16]
quantized_tflite_model = converter.convert()

# Evaluate TFLite model
quantized_accuracy = evaluate_tflite_model(quantized_tflite_model, X_test, y_test)

# Print accuracies
print("Original Model Accuracy: {:.2f}%".format(original_accuracy * 100))
print("Quantized Model Accuracy: {:.2f}%".format(quantized_accuracy * 100))

# Save the original model
model.save('model_awake.h5')
original_model_size = get_gzipped_model_size('model_awake.h5')
print("Size of gzipped original Keras model: %.2f Kbytes" % (original_model_size / 1024))

# Save the TFLite model
tflite_model_path = 'quantized_f32_model_awake.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(quantized_tflite_model)

# Calculate and print the size of the TFLite model
quantized_pruned_model_size = get_gzipped_model_size(tflite_model_path)
print("Size of gzipped quantized TFLite model: %.2f Kbytes" % (quantized_pruned_model_size / 1024))



Epoch 1/15
54/54 [==============================] - 1s 721us/step - loss: 0.7106 - accuracy: 0.6476
Epoch 2/15
54/54 [==============================] - 0s 570us/step - loss: 0.5598 - accuracy: 0.7240
Epoch 3/15
54/54 [==============================] - 0s 466us/step - loss: 0.5374 - accuracy: 0.7435
Epoch 4/15
54/54 [==============================] - 0s 474us/step - loss: 0.5258 - accuracy: 0.7455
Epoch 5/15
54/54 [==============================] - 0s 466us/step - loss: 0.5145 - accuracy: 0.7572
Epoch 6/15
54/54 [==============================] - 0s 458us/step - loss: 0.5053 - accuracy: 0.7594
Epoch 7/15
54/54 [==============================] - 0s 458us/step - loss: 0.4989 - accuracy: 0.7643
Epoch 8/15
54/54 [==============================] - 0s 452us/step - loss: 0.4930 - accuracy: 0.7688
Epoch 9/15
54/54 [==============================] - 0s 451us/step - loss: 0.4858 - accuracy: 0.7688
Epoch 10/15
54/54 [==============================] - 0s 453us/step - loss: 0.4766 - accuracy: 0.7750

INFO:tensorflow:Assets written to: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpjw_wdo3s/assets


INFO:tensorflow:Assets written to: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpjw_wdo3s/assets
2023-12-11 00:47:33.492260: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-11 00:47:33.492277: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


Original Model Accuracy: 76.33%
Quantized Model Accuracy: 76.33%
Size of gzipped original Keras model: 24.74 Kbytes
Size of gzipped quantized TFLite model: 8.27 Kbytes


2023-12-11 00:47:33.492398: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpjw_wdo3s
2023-12-11 00:47:33.493480: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-12-11 00:47:33.493488: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpjw_wdo3s
2023-12-11 00:47:33.496151: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-12-11 00:47:33.524831: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /var/folders/z_/800j_fm55b5fzs6d0ngk4_z80000gq/T/tmpjw_wdo3s
2023-12-11 00:47:33.531702: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 39304 microseconds.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.
